In [1]:
# dependencies
import re
import pandas as pd

In [2]:
# support methods

In [3]:
# main
raw = pd.read_parquet("../output/data.parquet")
data = pd.read_parquet("../output/data_formatted.parquet")

In [ ]:
datecols = [col for col in raw.columns if 'date' in col]
#for col in datecols: raw[col] = raw[col].astype('datetime64[ns]')
#raw.to_parquet("../output/data_formatted.parquet")

# Preview random record

In [4]:
data.sample().T

,209763
record_id,7936241
call_type,415E
description,DISTURBANCE-NOISE
report_created,N
location,37XX MAY ST
police_district,2
beat,2A
day_of_week,Sat
cleared_by,A
cleared_by_desc,POLICE MATTER RESOLVED AT SCENE


# Date range of records

In [6]:
data[datecols].min()

occurrence_date_pt   2019-01-01 00:00:00
received_date_pt     2019-01-01 00:00:00
dispatch_date_pt     2019-01-01 00:00:01
enroute_date_pt      2019-01-01 00:00:01
at_scene_date_pt     2019-01-01 00:00:01
clear_date_pt        2019-01-01 00:03:00
dtype: datetime64[ns]

In [7]:
data[datecols].max()

occurrence_date_pt   2024-07-09 23:58:00
received_date_pt     2024-07-09 23:58:00
dispatch_date_pt     2024-07-10 12:02:58
enroute_date_pt      2024-07-10 12:02:58
at_scene_date_pt     2024-07-10 12:20:20
clear_date_pt        2024-07-16 16:05:00
dtype: datetime64[ns]

# Rate of created reports

### overall

In [8]:
data.report_created.value_counts(
    ).to_frame().reset_index()

,report_created,count
0,N,1815447
1,Y,273288


In [9]:
data.report_created.value_counts(normalize=True
    ).to_frame().reset_index()

,report_created,proportion
0,N,0.869161
1,Y,0.130839


### by `description`

In [10]:
data[['report_created', 'description']].value_counts(
    ).to_frame().reset_index(
    ).head(30)

,report_created,description,count
0,N,DISTURBANCE-CLARIFY,248932
1,N,ADVISED ENTRY,166099
2,N,TRAFFIC STOP,147036
3,N,ALL UNITS BROADCAST,138262
4,N,INCOMPLETE CALL FOR POLICE,114175
5,N,WELFARE CHECK,104266
6,N,SUSPICIOUS SUBJECT/CIRCUMSTANCE-IN PROGRESS,67469
7,N,ALARM-SECURE NO EVID OF CRIME,46574
8,N,SUSPICIOUS VEHICLE-OCCUPIED,41805
9,N,ERRAND,37013


### by cleared description

In [11]:
data[['report_created', 'cleared_by_desc']].value_counts(
    ).to_frame().reset_index()

,report_created,cleared_by_desc,count
0,N,POLICE MATTER RESOLVED AT SCENE,1022102
1,N,CANCEL,501114
2,Y,POLICE MATTER RESOLVED AT SCENE,220482
3,N,AUTOMATICALLY (f...,219267
4,N,TRAFFIC CITATION,37467
5,N,OTHER/OUTSIDE AGENCY,30648
6,Y,OTHER/OUTSIDE AGENCY,27122
7,Y,PENDING,13163
8,Y,MISDEMEANOR CITATION,4277
9,Y,AUTOMATICALLY (f...,3806


### by `description` and `cleared_by_desc`

In [12]:
data[['report_created', 'description', 'cleared_by_desc']].value_counts(
    ).to_frame().reset_index(
    ).head(10)

,report_created,description,cleared_by_desc,count
0,N,DISTURBANCE-CLARIFY,POLICE MATTER RESOLVED AT SCENE,182419
1,N,ADVISED ENTRY,AUTOMATICALLY (f...,161961
2,N,ALL UNITS BROADCAST,CANCEL,131384
3,N,TRAFFIC STOP,POLICE MATTER RESOLVED AT SCENE,112450
4,N,WELFARE CHECK,POLICE MATTER RESOLVED AT SCENE,77489
5,N,INCOMPLETE CALL FOR POLICE,CANCEL,65433
6,N,DISTURBANCE-CLARIFY,CANCEL,64943
7,N,SUSPICIOUS SUBJECT/CIRCUMSTANCE-IN PROGRESS,POLICE MATTER RESOLVED AT SCENE,59950
8,N,INCOMPLETE CALL FOR POLICE,POLICE MATTER RESOLVED AT SCENE,47558
9,N,ALARM-SECURE NO EVID OF CRIME,POLICE MATTER RESOLVED AT SCENE,46497


# Call types

### overall

In [13]:
data[['call_type', 'description']].value_counts().head(30)

call_type  description                                
415        DISTURBANCE-CLARIFY                            267663
ADV        ADVISED ENTRY                                  166249
TSTOP      TRAFFIC STOP                                   155929
AU         ALL UNITS BROADCAST                            138445
952        INCOMPLETE CALL FOR POLICE                     115360
981        WELFARE CHECK                                  108328
927P       SUSPICIOUS SUBJECT/CIRCUMSTANCE-IN PROGRESS     74192
ALMSEC     ALARM-SECURE NO EVID OF CRIME                   46746
971        SUSPICIOUS VEHICLE-OCCUPIED                     45540
992E       ERRAND                                          38923
415DV      DISTURBANCE-DOMESTIC VIOLENCE-VERBAL ONLY       38677
SSTOP      SUBJECT STOP                                    34743
RPTNUM     REPORT NUMBER ASSIGNMENT                        33309
901        VEHICLE ACCIDENT-NO OR UNKNOWN INJURIES         31680
415F       DISTURBANCE-FAMILY      

### SHOTSPOTTER

In [14]:
shotspotter = data.description.str.contains("SHOTSPOTTER",)

In [15]:
data.loc[shotspotter, ['description',
                       'report_created',
                       'cleared_by_desc']
].value_counts().to_frame().reset_index()

,description,report_created,cleared_by_desc,count
0,SHOTSPOTTER,N,POLICE MATTER RESOLVED AT SCENE,2990
1,SHOTSPOTTER,Y,POLICE MATTER RESOLVED AT SCENE,1057
2,SHOTSPOTTER,N,CANCEL,874
3,SHOTSPOTTER,N,OTHER/OUTSIDE AGENCY,14
4,SHOTSPOTTER,Y,CANCEL,3
5,SHOTSPOTTER,Y,MISDEMEANOR CITATION,3
6,SHOTSPOTTER,N,AUTOMATICALLY (f...,2


In [17]:
shotspotter_rate = f"The data indicate there was a `report_created` for \
{data.loc[shotspotter, 'report_created'
].value_counts(normalize=True)['Y']*100:.1f}% of ShotSpotter events \
({data.loc[shotspotter, 'report_created'
].value_counts()['Y']:,} of {shotspotter.sum():,} records). "

overall_rate = f"The data indicate there was a `report_created` for \
{data.report_created.value_counts(normalize=True)['Y']*100:.1f}% of all events \
({data.report_created.value_counts()['Y']:,} of {data.shape[0]:,} records)."

print(shotspotter_rate)
print(overall_rate)

The data indicate there was a `report_created` for 21.5% of ShotSpotter events (1,063 of 4,943 records). 
The data indicate there was a `report_created` for 13.1% of all events (273,288 of 2,088,735 records).


In [18]:
shotspotter_rate = f"There is a reported `at_scene_date_pt` for \
{data.loc[shotspotter, 'at_scene_date_pt'
].notna().value_counts(normalize=True)[True]*100:.1f}% of ShotSpotter events \
({data.loc[shotspotter, 'at_scene_date_pt'
].notna().value_counts()[True]:,} of {shotspotter.sum():,} records). "

overall_rate = f"There is a reported `at_scene_date_pt` for \
{data.at_scene_date_pt.notna().value_counts(normalize=True)[True]*100:.1f}% of all events \
({data.at_scene_date_pt.notna().value_counts()[True]:,} of {data.shape[0]:,} records)."

print(shotspotter_rate)
print(overall_rate)

There is a reported `at_scene_date_pt` for 82.0% of ShotSpotter events (4,053 of 4,943 records). 
There is a reported `at_scene_date_pt` for 60.6% of all events (1,265,733 of 2,088,735 records).


In [19]:
data.loc[data.description.str.contains("SHOT", flags=re.I),
'description'].value_counts()

description
SHOTS FIRED - LESS THAN 15 AGO    13043
SHOTSPOTTER                        4943
Name: count, dtype: int64

### What would it look like for actual gunfire to have been identified/reported in one of these events?

In [20]:
data.loc[data.description.str.contains("SHOT", flags=re.I),
['report_created', 'description', 'cleared_by_desc']].value_counts().to_frame().reset_index()

,report_created,description,cleared_by_desc,count
0,N,SHOTS FIRED - LESS THAN 15 AGO,CANCEL,6819
1,N,SHOTS FIRED - LESS THAN 15 AGO,POLICE MATTER RESOLVED AT SCENE,5776
2,N,SHOTSPOTTER,POLICE MATTER RESOLVED AT SCENE,2990
3,Y,SHOTSPOTTER,POLICE MATTER RESOLVED AT SCENE,1057
4,N,SHOTSPOTTER,CANCEL,874
5,Y,SHOTS FIRED - LESS THAN 15 AGO,POLICE MATTER RESOLVED AT SCENE,350
6,N,SHOTS FIRED - LESS THAN 15 AGO,AUTOMATICALLY (f...,65
7,Y,SHOTS FIRED - LESS THAN 15 AGO,CANCEL,18
8,N,SHOTSPOTTER,OTHER/OUTSIDE AGENCY,14
9,N,SHOTS FIRED - LESS THAN 15 AGO,OTHER/OUTSIDE AGENCY,12


In [21]:
data.loc[data.cleared_by_desc.str.contains("automatically", na=False, flags=re.I),
'cleared_by_desc'].value_counts()

cleared_by_desc
AUTOMATICALLY                               (for advised entries)    223073
Name: count, dtype: int64

- "MISDEMEANOR CITATION"? Or "POLICE MATTER RESOLVED AT SCENE"? Some of these appear to have had a report created.
    - If "POLICE MATTER RESOLVED AT SCENE" with a `report_created` can include actual gunfire, what would that mean about how "RESOLVED" is used in this label? And in others? (seems a little too hand-wavy or protective over key details about distinct scenarios)
- What warrants "CANCEL"? Or an "AUTOMATICALLY" cleared event "(for advised entries)"?
- "POLICE MATTER RESOLVED AT SCENE"?
- All the "SHOTS FIRED" calls include the phrase "LESS THAN 15 AGO." What's that about?

### Missing persons

In [22]:
missper = data.description.str.contains("MISSING PERSON")

In [23]:
data.loc[missper, ['description',
                       'report_created',
                       'cleared_by_desc']
].value_counts().to_frame().reset_index()

,description,report_created,cleared_by_desc,count
0,AT-RISK MISSING PERSON,N,POLICE MATTER RESOLVED AT SCENE,5812
1,AT-RISK MISSING PERSON,N,CANCEL,4561
2,MISSING PERSON-FAXED REPORT,Y,POLICE MATTER RESOLVED AT SCENE,3015
3,MISSING PERSON-FAXED REPORT,Y,OTHER/OUTSIDE AGENCY,2705
4,MISSING PERSON-REPORT,Y,PENDING,2468
5,AT-RISK MISSING PERSON,Y,POLICE MATTER RESOLVED AT SCENE,1615
6,MISSING PERSON-REPORT,Y,POLICE MATTER RESOLVED AT SCENE,1579
7,AT-RISK MISSING PERSON,Y,PENDING,1547
8,AT-RISK MISSING PERSON,N,PENDING,1392
9,MISSING PERSON-REPORT,N,POLICE MATTER RESOLVED AT SCENE,1124


In [24]:
data.loc[missper, 'report_created'].value_counts(normalize=True)

report_created
Y    0.513301
N    0.486699
Name: proportion, dtype: float64

In [25]:
missper_rate = f"The data indicate there was a `report_created` for \
{data.loc[missper, 'report_created'
].value_counts(normalize=True)['Y']*100:.1f}% of MISSING PERSON events \
({data.loc[missper, 'report_created'
].value_counts()['Y']:,} of {missper.sum():,} records). "

overall_rate = f"The data indicate there was a `report_created` for \
{data.report_created.value_counts(normalize=True)['Y']*100:.1f}% of all events \
({data.report_created.value_counts()['Y']:,} of {data.shape[0]:,} records)."

print(missper_rate)
print(overall_rate)

The data indicate there was a `report_created` for 51.3% of MISSING PERSON events (15,784 of 30,750 records). 
The data indicate there was a `report_created` for 13.1% of all events (273,288 of 2,088,735 records).


In [26]:
missper_rate = f"There is a reported `at_scene_date_pt` for \
{data.loc[missper, 'at_scene_date_pt'
].notna().value_counts(normalize=True)[True]*100:.1f}% of MISSING PERSON events \
({data.loc[missper, 'at_scene_date_pt'
].notna().value_counts()[True]:,} of {missper.sum():,} records). "

overall_rate = f"There is a reported `at_scene_date_pt` for \
{data.at_scene_date_pt.notna().value_counts(normalize=True)[True]*100:.1f}% of all events \
({data.at_scene_date_pt.notna().value_counts()[True]:,} of {data.shape[0]:,} records)."

print(missper_rate)
print(overall_rate)

There is a reported `at_scene_date_pt` for 31.9% of MISSING PERSON events (9,804 of 30,750 records). 
There is a reported `at_scene_date_pt` for 60.6% of all events (1,265,733 of 2,088,735 records).


In [27]:
data.loc[data.description.str.contains("MISSING", flags=re.I),
'description'].value_counts()

description
AT-RISK MISSING PERSON         16489
MISSING PERSON-REPORT           8439
MISSING PERSON-FAXED REPORT     5822
Name: count, dtype: int64